In [1]:
import pandas as pd
import requests
import os

In [2]:
api_token = 'P4MR7PELIC3KHQW4FLHC'

In [3]:
request = requests.get('https://www.eventbriteapi.com/v3/events/68291233919/?expand=venue,organizer,format,category,subcategory,bookmark_info,refund_policy,ticket_classes,ticket_availability,external_ticketing,music_properties,publish_settings,guestlist_metrics,listing_properties&token='+api_token)
request_json = request.json()

In [4]:
print('Name: %s\nOrganizer:    %s\nOrganization: %s' % (request_json['name']['text'],request_json['organizer_id'],request_json['organization_id']))

Name: Therapeutic Yoga
Organizer:    18291367526
Organization: 283974720005


In [5]:
request = requests.get('https://www.eventbriteapi.com/v3/events/68005132181/?expand=venue,organizer,format,category,subcategory,bookmark_info,refund_policy,ticket_classes,ticket_availability,external_ticketing,music_properties,publish_settings,guestlist_metrics,listing_properties&token='+api_token)
request_json = request.json()

In [6]:
print('Name: %s\nOrganizer:    %s\nOrganization: %s' % (request_json['name']['text'],request_json['organizer_id'],request_json['organization_id']))

Name: Restorative Yoga
Organizer:    18291367526
Organization: 283974720005


In [7]:
request = requests.get('https://www.eventbriteapi.com/v3/events/64006448009/?expand=venue,organizer,format,category,subcategory,bookmark_info,refund_policy,ticket_classes,ticket_availability,external_ticketing,music_properties,publish_settings,guestlist_metrics,listing_properties&token='+api_token)
request_json = request.json()

In [8]:
print('Name: %s\nOrganizer:    %s\nOrganization: %s' % (request_json['name']['text'],request_json['organizer_id'],request_json['organization_id']))

Name: Yoga at the Library
Organizer:    7793159909
Organization: 130884443347


In [9]:
request_json

{'name': {'text': 'Yoga at the Library', 'html': 'Yoga at the Library'},
 'description': {'text': 'Join the East Hampton Library as we welcome Registered Nurse Andrea Siegel as she introduces us to a Yoga program for adults which includes chair yoga! Andrea Siegel has 20 years of nursing experience and has been practicing yoga for over 20 years and is certified in chair yoga. She has struggled with back problems both genetically and through injury and has found yoga to be not only helpful in pain relief but also beneficial in strengthening muscles, improving flexibility and maintaining balance.',
  'html': '<P><SPAN>Join the East Hampton Library as we welcome Registered Nurse Andrea Siegel as she introduces us to a Yoga program for adults which includes chair yoga! Andrea Siegel has 20 years of nursing experience and has been practicing yoga for over 20 years and is certified in chair yoga. She has struggled with back problems both genetically and through injury and has found yoga to b

In [10]:
request = requests.get('https://www.eventbriteapi.com/v3/events/search/?categories=107&subcategories=7005&expand=ticket_classes&location.address=newyork&expand=ticket_classes&token='+api_token)
request_json = request.json()
request_json

{'pagination': {'object_count': 169,
  'page_number': 1,
  'page_size': 50,
  'page_count': 4,
  'has_more_items': True},
 'events': [{'name': {'text': "Meditation as Medication: A Doctor's Perspective",
    'html': 'Meditation as Medication: A Doctor&#39;s Perspective'},
   'description': {'text': "Can meditation improve high blood pressure, anxiety, depression, or pain? \xa0\r\n\r\nIf a doctor told you to meditate, would you hesitate?\r\nScience of Spirituality Presents\r\nMeditation as Medication:\xa0A Doctor's Perspective\r\nMeet with top Medical Experts and Researchers and learn how meditation has changed them and their patients. \xa0Both speakers are co-authors of the book - Meditation as Medication, a best seller!\r\nRetreat is FREE of Charge\r\nSaturday, September 28, 2019\xa0\r\n10.30 AM – Registration & Welcome\r\n11 AM – 12.30 PM\r\nMeditation for Body, Mind and Soul: Meditation Experiences within Clinical Practices\r\nSpeaker: Dr. Matt Raider, M.D., Medical Director, CT\r\n

In [11]:
request_json['pagination']

{'object_count': 169,
 'page_number': 1,
 'page_size': 50,
 'page_count': 4,
 'has_more_items': True}

In [12]:
yoga_df = pd.DataFrame()

In [13]:
event_properties = [{'event_name':event['name']['text'],
                     'organizer':event['organizer_id'],
                     'organization':event['organization_id'],
                     'price':event['ticket_classes'][0]['cost']} 
                    for event in request_json['events']]

In [14]:
yoga_df = pd.DataFrame(event_properties)

In [15]:
yoga_df.price.fillna(value=0,inplace=True)

In [16]:
yoga_df.iloc[0]

event_name      Meditation as Medication: A Doctor's Perspective
organizer                                             2583029518
organization                                         38243310276
price                                                          0
Name: 0, dtype: object

In [17]:
yoga_df.price[0]

0

In [18]:
for row,value in enumerate(yoga_df.price):
    if value != 0:
        yoga_df.price[row] = float(value['major_value'])

In [19]:
yoga_df

,event_name,organizer,organization,price
0,Meditation as Medication: A Doctor's Perspective,2583029518,38243310276,0
1,Free Yoga Class: Yoga Basics and Breath,15386741481,218201622803,0
2,Yoga in the Garden 2019,2339390903,33673116301,0
3,Yoga in the Fort Greene Park,18937400320,293950156663,0
4,DRUNK YOGA® at Row NYC...FREE Wine! Saturdays ...,15821524808,236398958683,45
5,DUMBO SOCIAL INFLUENCER EVENT (+ barre class!)...,12054388158,166625625478,0
6,Advanced Yogalates,18291367526,283974720005,100
7,ViniYoga,18291367526,283974720005,100
8,Tune In - Restorative Yoga & Himalayan Singing...,20312271518,228941257663,45
9,Rooftop Yoga for People of Color,17989031888,277085064664,20
